In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')
from datetime import date, timedelta
#!pip install twint
import twint
import nest_asyncio
nest_asyncio.apply()
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [6]:
c = twint.Config()
c.Username = "detikcom"
c.Search = '"berita" lang:id'
c.Limit = 10000
c.Store_csv = True
c.Output = 'new2.csv'

twint.run.Search(c)

1577127919067041793 2022-10-04 09:46:12 +0700 <detikcom> "Itu bisa dipastikan berita yang tidak benar dan hoax yang sama sekali sumbernya tidak jelas. Kami tegaskan bahwa pertemuan itu tidak ada," kata Nurul Arifin.  https://t.co/QEHVFKl6QX
1576881836382363649 2022-10-03 17:28:21 +0700 <detikcom> "Intinya mah saya mengucapkan selamat, setiap berita bahagia hadir kepada sahabat-sahabat kan kita harus ikut bahagia. Pak Anies adalah sahabat saya, jadi saya ucapkan selamat," kata Ridwan Kamil. #Pemilu2024  https://t.co/9XDRLVLToo
1576812589694205955 2022-10-03 12:53:11 +0700 <detikcom> Publik dihebohkan dengan kabar Lesti Kejora yang melaporkan Rizky Billar karena KDRT. Ada juga kabar dari Denise Chariesta yang jadi selingkuhan. Ini deretan berita terpopuler sepekan:   https://t.co/Sgy8k4wBEV
1576587425220108289 2022-10-02 21:58:28 +0700 <detikcom> Berita terpopuler pekan ini datang dari curhatan istri yang memergoki suaminya selingkuh melalui aplikasi media sosial Michat. Simak ceritanya:

1567776159437307911 2022-09-08 14:25:38 +0700 <detikcom> Fakta baru terungkap dalam kasus dokter bunuh diri di Malang. Korban diketahui melakukan aksinya di rumah kontrakan milik temannya. Simak berita selengkapnya.  via @detik_jatim  https://t.co/ssNMJKTn3t
1567569841262845954 2022-09-08 00:45:48 +0700 <detikcom> Barcelona mengawali kiprahnya di Liga Champions 2022/2023 dengan menghadapi Viktoria Plzen. Berita ini memuat link live streaming Barcelona vs Viktoria Plzen.  https://t.co/nr49HLpbCe
1567418881878200321 2022-09-07 14:45:57 +0700 <detikcom> Mayat ini ditemukan di dalam tas merah dalam kondisi membusuk. Polisi menduga, mayat ini sudah meninggal lebih dari dua hari. Simak berita selengkapnya.  via @detik_jatim  https://t.co/qjuKzQD40F
1567060323261157376 2022-09-06 15:01:10 +0700 <detikcom> Berdasarkan hasil pemeriksaan dan olah TKP, polisi telah mengantongi nama dua terduga pelaku. Simak berita selengkapnya.  via @detik_jatim  https://t.co/rrk2za0xFx
1567013704478216195 2022-0

In [7]:
tweets_data = pd.read_csv('new2.csv')
tweets = tweets_data[['id', 'username', 'tweet']]
tweets

,id,username,tweet
0,1577127919067041793,detikcom,"""Itu bisa dipastikan berita yang tidak benar d..."
1,1576881836382363649,detikcom,"""Intinya mah saya mengucapkan selamat, setiap ..."
2,1576812589694205955,detikcom,Publik dihebohkan dengan kabar Lesti Kejora ya...
3,1576587425220108289,detikcom,Berita terpopuler pekan ini datang dari curhat...
4,1576530961587642368,detikcom,Kerusuhan yang terjadi setelah laga Arema FC m...
5,1576496814076968960,detikcom,Sejumlah media asing menyoroti tragedi di Stad...
6,1576400117896077312,detikcom,Tragedi Kanjuruhan setelah Arema FC vs Perseba...
7,1576399322127761409,detikcom,Tragedi Kanjuruhan setelah Arema FC vs Perseba...
8,1576376014774874112,detikcom,#THREAD - Sebanyak 127 orang menjadi korban te...
9,1576289911249985539,detikcom,Teror pamer kelamin menghantui mahasiswi di se...
